In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly import tools
import plotly.plotly as py

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import datetime
import time
import pandas_datareader as pdr

In [3]:
df=pd.read_csv("/data/insiderdata.csv")
df=df.drop('Unnamed: 0.1',1)# remove unnecessary columns

In [4]:
df.head()

,Unnamed: 0,Buy/Sell,Transaction Date,Acceptance Date Time,Issuer Name,Issuer Trading Symbol,Reporting Owner Name,Reporting Owner Relationship,Transcation Share,Price Per share,Total Value,Shares Owned Following Transaction,Form,sector,market_cap
0,0,Sell,2017-01-04,2017-01-06 20:57:22,Immune Design Corp.,IMDZ,Brady Stephen R,officer,"1,642",$5.3500,"$8,784.7000","30,490",Form 4,Health Care,282403939.76
1,1,Sell,2017-01-05,2017-01-06 20:46:33,HURCO COMPANIES INC,HURC,Volovic Gregory S,officer,"2,966",$33.9000,"$100,547.0000","28,839",Form 4,Capital Goods,279583548.51
2,2,Sell,2017-01-06,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,234,$33.7000,"$7,885.8000","17,353",Form 4,Capital Goods,279583548.51
3,3,Sell,2017-01-04,2017-01-06 20:46:02,HURCO COMPANIES INC,HURC,McClelland Sonja K,officer,372,$34.3000,"$12,759.6000","12,836",Form 4,Capital Goods,279583548.51
4,4,Sell,2017-01-05,2017-01-06 20:45:30,HURCO COMPANIES INC,HURC,Donlon John P.,officer,"1,018",$33.9000,"$34,510.2000","12,689",Form 4,Capital Goods,279583548.51


In [5]:
df=df.drop("Form",1)
df['Issuer Trading Symbol']=[str(x).upper() for x in df['Issuer Trading Symbol']] #would be useful for matching up 
df['Transaction Date']=pd.to_datetime(df['Transaction Date'],errors='coerce')

In [6]:
df['Transcation Share']=[x.replace(",","") for x in df['Transcation Share']]
df['Price Per share']=[x.replace(",","") for x in df['Price Per share']]
df['Price Per share']=[x.replace("$","") for x in df['Price Per share']]

In [7]:
df['Transcation Share']=pd.to_numeric(df['Transcation Share'])
df["Price Per share"]=pd.to_numeric(df['Price Per share'])

In [8]:
df_buy=df[df['Buy/Sell']=="Buy"]
df_sell=df[df['Buy/Sell']=="Sell"]
print("Buy",df_buy.shape)
print("Sell",df_sell.shape)

Buy (12427, 14)
Sell (17573, 14)


In [9]:
df_count=df.groupby(df['Transaction Date']).count()
df_buy_count=df_buy.groupby(df_buy['Transaction Date']).count()
df_sell_count=df_sell.groupby(df_sell['Transaction Date']).count()

In [10]:
trace = go.Scatter(
                x=df_count.index,
                y=df_count['Unnamed: 0'],
                name = "overall activity",
                line = dict(color = '#000000'),
                opacity = 0.5)

trace_buy = go.Scatter(
                x=df_buy_count.index,
                y=df_buy_count['Unnamed: 0'],
                name = "Buy",
                line = dict(color = '#008000'),
                opacity = 0.8)

trace_sell = go.Scatter(
                x=df_sell_count.index,
                y=df_sell_count['Unnamed: 0'],
                name = "Sell",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

data = [trace,trace_buy,trace_sell]

layout = dict(
    title = "Insider Activity Over Time",
    yaxis=dict(title='Number of Incidents')
    
)

fig = dict(data=data, layout=layout)
iplot(fig)

In [11]:
trace = go.Scatter(
                x=df_count.index,
                y=df_count['Unnamed: 0'],
                name = "Overall Insider activity",
                line = dict(color = '#000000'),
                opacity = 0.5)

trace_buy = go.Scatter(
                x=df_buy_count.index,
                y=df_buy_count['Unnamed: 0'],
                name = "Buy",
                line = dict(color = '#008000'),
                opacity = 0.8)

trace_sell = go.Scatter(
                x=df_sell_count.index,
                y=df_sell_count['Unnamed: 0'],
                name = "Sell",
                line = dict(color = '#FF0000'),
                opacity = 0.8)

data = [trace,trace_buy,trace_sell]

layout = dict(
    title = "Insider Activity Over Time",
    xaxis = dict(
        range = ['2016-10-10','2017-01-08']),
    yaxis=dict(
     title="Number of Insider Incidents")
)

fig = dict(data=data, layout=layout)
iplot(fig)

# Distribution Of number of Insiders Per company

In [12]:
def insiders_per_company(df):
    company=[]
    insider_count=[]
    for x in range(len(df['Issuer Name'].unique())):
        company.append(df['Issuer Name'].unique()[x])
        insider_count.append(len(df[df['Issuer Name']==df['Issuer Name'].unique()[x]]['Reporting Owner Name'].unique()))
    df_company=pd.DataFrame(company)
    df_company["insider_count"]=insider_count
    df_company.columns=['company',"insider_count"]
    dist=pd.DataFrame(df_company['insider_count'].value_counts())
    dist.columns=['count']
    return dist
    

In [13]:
dist=insiders_per_company(df)
dist_buy=insiders_per_company(df_buy)
dist_sell=insiders_per_company(df_sell)

In [14]:
trace0 = go.Bar(
    x=dist.index,
    y=dist['count'],
    name='overall distribution',
    marker=dict(
        color='#96d9d6'
    )
)

trace2 = go.Scatter(
    x = dist_buy.index,
    y = dist_buy['count'],
    name='buy',
    mode = 'lines+markers',
    marker = dict(
        size = 5,
        color = '#735797'
    )
)
trace3 = go.Scatter(
    x = dist_sell.index,
    y = dist_sell['count'],
    name='sell',
    mode = 'lines+markers',
    marker = dict(
        size = 5,
        color = '#7ac74c'
    )
)
data=[trace0,trace2,trace3]
layout = dict(title = 'Distribution of number of insiders per company',
              yaxis = dict(title='Number of Companies'),
              xaxis = dict(title='Number of insiders')
             )
fig = dict(data=data, layout=layout)
iplot(fig)


# Distribution of number of Transactions per Insider

In [15]:
def transactions_per_insider(df):
    trans_per_insider=pd.DataFrame(df['Reporting Owner Name'].value_counts())
    trans_per_insider=trans_per_insider.reset_index()
    trans_per_insider.columns=['Name','trans_num']
    trans=trans_per_insider.groupby(trans_per_insider['trans_num']).count()
    trans.columns=['count']
    return trans
        
    

In [16]:
trans=transactions_per_insider(df)
trans_buy=transactions_per_insider(df_buy)
trans_sell=transactions_per_insider(df_sell)


In [17]:
transac=trans[trans.index<50]
buy=trans_buy[trans_buy.index<50]
sell=trans_sell[trans_sell<50]

In [18]:
trace0 = go.Bar(
    x=transac.index,
    y=transac['count'],
    name='overall distribution',
    marker=dict(
        color='#96d9d6'
    )
)

trace2 = go.Scatter(
    x = buy.index,
    y = buy['count'],
    name='buy',
    mode = 'lines+markers',
    marker = dict(
        size = 5,
        color = '#735797'
    )
)
trace3 = go.Scatter(
    x = sell.index,
    y = sell['count'],
    name='sell',
    mode = 'lines+markers',
    marker = dict(
        size = 5,
        color = '#7ac74c'
    )
)
data=[trace0,trace2,trace3]
layout = dict(title = 'Distribution of number of transactions per insider',
              yaxis = dict(title='Number of insiders'),
              xaxis = dict(title='Number of transactions')
             )
fig = dict(data=data, layout=layout)
iplot(fig)


# Top Companies for Insider activities

In [19]:
company=pd.DataFrame(df['Issuer Name'].value_counts())
company.columns=['incidents_num']
top_company=company[company['incidents_num']>100]


In [20]:
trace=go.Bar(
x=top_company.index,
y=top_company['incidents_num'],

    marker=dict(
        color='#e88372',
        line=dict(
            color='#e15c46',
            width=1.5,
        )
    ),
    opacity=0.8
)
data = [trace]
layout = go.Layout(
    title='Top companies for insider activities',
    xaxis = dict(title = 'Companies'),
    yaxis = dict(title = 'Total Number of Insider Incidents'),
    margin=go.layout.Margin(
        l=50,
        r=100,
        b=200,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Market Cap of some of the top Companies

In [21]:
a=list(top_company.index)
m=[]
for x in range(len(a)):
    f=df[df['Issuer Name']==a[x]]
    d=list(f['market_cap'])[0]
    m.append(d)

In [22]:
top_market_cap=pd.DataFrame(a)
top_market_cap.columns=['Company']
top_market_cap['market_cap']=m
top_market_cap=top_market_cap[top_market_cap['market_cap']!='unknown']
top_market_cap['market_cap']=[float(x) for x in top_market_cap['market_cap']]

In [23]:
trace=go.Bar(
x=top_market_cap['Company'],
y=top_market_cap['market_cap'],

    marker=dict(
        color='#dfffd1',
        line=dict(
            color='#4C9900',
            width=1.5,
        )
    ),
    opacity=0.8
)
data = [trace]
layout = go.Layout(
    title='Market Cap for Top companies',
    xaxis = dict(title = 'Companies'),
    yaxis = dict(title = 'Market Cap in Billions USD'),
    width=800,
    margin=go.layout.Margin(
        l=70,
        r=120,
        b=300,
        t=100,
        pad=4
    )
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Insider activity for different Roles

In [24]:
df_insider=pd.crosstab(df['Transaction Date'], df['Reporting Owner Relationship'])

In [25]:
ten_per_owner = go.Scatter(
                x=df_insider.index,
                y=df_insider[' 10% owner'],
                name = " 10% owner",
                line = dict(color = '#000000'),
                opacity = 0.5)
ten_per_owner_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' 10% owner other'],
                name = " 10% owner other",
                line = dict(color = '#533965'),
                opacity = 0.8)
director = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director'],
                name = "director",
                line = dict(color = '#56ac4d'),
                opacity = 0.8)
director_ten_per_owner = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director 10% owner'],
                name = "director 10% owner",
                line = dict(color = '#c46c71'),
                opacity = 0.8)
director_ten_per_owner_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director 10% owner other'],
                name = "director 10% owner other",
                line = dict(color = '#813338'),
                opacity = 0.8)
director_officer = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director officer'],
                name = "director officer",
                line = dict(color = '#0fd2ee'),
                opacity = 0.8)

director_officer_ten_per_owner = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director officer 10% owner'],
                name = "director officer 10% owner",
                line = dict(color = '#008080'),
                opacity = 0.8)
director_officer_ten_per_owner_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director officer 10% owner other'],
                name = "director officer 10% owner other",
                line = dict(color = '#e15c46'),
                opacity = 0.8)
director_officer_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director officer other'],
                name = "director officer other",
                line = dict(color = '#f6e20e'),
                opacity = 0.8)
director_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' director other'],
                name = "director other",
                line = dict(color = '#641E16'),
                opacity = 0.8)
officer = go.Scatter(
                x=df_insider.index,
                y=df_insider[' officer'],
                name = "officer",
                line = dict(color = '#45B39D'),
                opacity = 0.8)
officer_ten_per_owner = go.Scatter(
                x=df_insider.index,
                y=df_insider[' officer 10% owner'],
                name = "officer 10% owner",
                line = dict(color = '#FF00FF'),
                opacity = 0.8)
officer_ten_per_owner_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' officer 10% owner other'],
                name = "officer 10% owner other",
                line = dict(color = '#808000'),
                opacity = 0.8)
officer_other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' officer other'],
                name = "officer other",
                line = dict(color = '#5233FF'),
                opacity = 0.8)
other = go.Scatter(
                x=df_insider.index,
                y=df_insider[' other'],
                name = "other",
                line = dict(color = '#FF3349'),
                opacity = 0.8)




data=[ten_per_owner,ten_per_owner_other,director,director_ten_per_owner,director_ten_per_owner_other,director_officer,
      director_officer_ten_per_owner,director_officer_ten_per_owner_other,director_officer_other,director_other,
     officer,officer_ten_per_owner,officer_ten_per_owner_other,officer_other,other]

layout = dict(
    title = "Insider Activity for different Roles",
    xaxis = dict(
        range = ['2016-10-10','2017-01-08']),
    yaxis=dict(
     title="Number of Insider Incidents")
    
)

fig = dict(data=data, layout=layout)
iplot(fig)

# Insider activity for different Sectors

In [26]:
df1=df[df['sector']!='unknown']

In [27]:
df_sector=pd.crosstab(df1['Transaction Date'],df1['sector'])


In [28]:
basic_industries = go.Scatter(
                x=df_sector.index,
                y=df_sector['Basic Industries'],
                name = "Basic Industries",
                line = dict(color = '#8dbea6'),
                opacity = 0.8)
capital_goods = go.Scatter(
                x=df_sector.index,
                y=df_sector['Capital Goods'],
                name = "Capital Goods",
                line = dict(color = '#f1d6cc'),
                opacity = 0.8)
consumer_durables = go.Scatter(
                x=df_sector.index,
                y=df_sector['Consumer Durables'],
                name = "Consumer Durables",
                line = dict(color = '#3ab286'),
                opacity = 0.8)
consumer_services = go.Scatter(
                x=df_sector.index,
                y=df_sector['Consumer Services'],
                name = "Consumer Services",
                line = dict(color = '#dde4e5'),
                opacity = 0.8)
energy = go.Scatter(
                x=df_sector.index,
                y=df_sector['Energy'],
                name = "Energy",
                line = dict(color = '#3aa2b2'),
                opacity = 0.8)
finance = go.Scatter(
                x=df_sector.index,
                y=df_sector['Finance'],
                name = "Finance",
                line = dict(color = '#c45b1a'),
                opacity = 0.8)
health_care = go.Scatter(
                x=df_sector.index,
                y=df_sector['Health Care'],
                name = "Health Care",
                line = dict(color = '#591b1b'),
                opacity = 0.8)
miscellaneous = go.Scatter(
                x=df_sector.index,
                y=df_sector['Miscellaneous'],
                name = "Miscellaneous",
                line = dict(color = '#3a66b2'),
                opacity = 0.8)
public_utilities = go.Scatter(
                x=df_sector.index,
                y=df_sector['Public Utilities'],
                name = "Public Utilities",
                line = dict(color = '#d3b68d'),
                opacity = 0.8)
transportation = go.Scatter(
                x=df_sector.index,
                y=df_sector['Transportation'],
                name = "Transportation",
                line = dict(color = '#d83232'),
                opacity = 0.8)
technology = go.Scatter(
                x=df_sector.index,
                y=df_sector['Technology'],
                name = "Technology",
                line = dict(color = '#84b655'),
                opacity = 0.8)

data=[technology,transportation,health_care,consumer_durables,consumer_services,miscellaneous,energy,public_utilities,finance,capital_goods,basic_industries]
layout = dict(
    title = "Insider Activity for different Sectors",
    xaxis = dict(
        range = ['2016-10-10','2017-01-08']),
    yaxis=dict(
     title="Number of Insider Incidents")
    
)
fig = dict(data=data, layout=layout)
iplot(fig)

# Insider Selling activity vs SP500

In [29]:
sp500=pd.read_csv('/data/sp500.csv')
sp500['Date']=pd.to_datetime(sp500['Date'],errors='coerce')

In [30]:
grouped=df_sell.groupby(df_sell['Transaction Date']).sum()
trace0 = go.Bar(
                x=grouped.index,
                y=grouped['Transcation Share'],
                name = "Transaction Share",
                marker = dict(color = '#008000'),
                )
trace1 = go.Scatter(
                x=sp500['Date'],
                y=sp500['Volume'],
                name = "sp500",
                line = dict(color = '#f6e20e'),
                opacity = 0.8,
                yaxis='y2')

data = [trace0,trace1]

layout = dict(
    title = "Insider Selling Activity vs SP500",
    yaxis=dict(
        title='Insider Transaction volume'
    ),
    yaxis2=dict(
        title='sp500 volume',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    ),
    xaxis = dict(
        range = ['2016-09-25','2017-01-12'])
    
)

fig = dict(data=data, layout=layout)
iplot(fig)

#  Analyzing Change in Short term stock prices after insider activity

In [31]:
sym=['GOOG','FB','CRM','FBP','SNX','CPF','HPE','KLIC','SGA']
top=df[df['Issuer Trading Symbol'].isin(sym)]

In [32]:
def calculate_future_prices(a):
    x=top[top['Issuer Trading Symbol']==a]
    
    sp=pdr.get_data_yahoo(a, 
                          start=datetime.datetime(2016, 8, 18), 
                          end=datetime.datetime(2018, 12, 26))
    
    x['new_trans_date']=[time.strptime(str(y.date()), "%Y-%m-%d") for y in x['Transaction Date']]
    sp.index=[time.strptime(str(x.date()), "%Y-%m-%d") for x in sp.index]
    x=x.reset_index()
    
    l,m,n,o,p,q,v,u,r=([] for i in range (9))
    for i in range(len(x)):
        for j in range(len(sp)):
            if x['new_trans_date'][i]==sp.index[j]:
                
                q.append(sp['Adj Close'][j])
                l.append(sp['Adj Close'][j+1])
                m.append(sp['Adj Close'][j+2])
                n.append(sp['Adj Close'][j+3])
                o.append(sp['Adj Close'][j+4])
                p.append(sp['Adj Close'][j+5])
                r.append(sp['Adj Close'][j+30])
                u.append(sp['Adj Close'][j+180])
                v.append(sp['Adj Close'][j+365])
                
    x['Adj Close']=q
    x['Adj Close day1']=l
    x['Adj Close day2']=m
    x['Adj Close day3']=n
    x['Adj Close day4']=o
    x['Adj Close day5']=p
    x['Adj Close month']=r
    x['Adj Close 6month']=u
    x['Adj Close year']=v
    
    return x
    

In [ ]:
goog=calculate_future_prices("GOOG")
crm=calculate_future_prices("CRM")
fb=calculate_future_prices("FB")
fbp=calculate_future_prices("FBP")
snx=calculate_future_prices("SNX")
cpf=calculate_future_prices("CPF")
klic=calculate_future_prices("KLIC")


In [33]:
# pandas datreader won't work always, in that case you can use my predownloaded data
"""goog=pd.read_csv("/data/yahoo_finance_data/goog.csv")
crm=pd.read_csv("/data/yahoo_finance_data/crm.csv")
fb=pd.read_csv("/data/yahoo_finance_data/fb.csv")
fbp=pd.read_csv("/data/yahoo_finance_data/fbp.csv")
snx=pd.read_csv("/data/yahoo_finance_data/snx.csv")
cpf=pd.read_csv("/data/yahoo_finance_data/cpf.csv")
klic=pd.read_csv("/data/yahoo_finance_data/klic.csv")"""

In [34]:
x=pd.concat([goog,crm,fb,fbp,snx,cpf,klic])
x=x.drop(labels=['level_0','index','Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'],axis=1)


In [35]:
x=x.reset_index()

In [36]:
x['Price Per share']=[y.replace(",","") for y in x['Price Per share']]
x['Price Per share']=[y.replace("$","") for y in x['Price Per share']]
x["Price Per share"]=pd.to_numeric(x['Price Per share'])

In [37]:
def calculate_returns(x):
    a,b,c,d,e,f,g,h=([] for k in range(8))
    for z in range(len(x)):
        a.append(((x['Adj Close day1'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        b.append(((x['Adj Close day2'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        c.append(((x['Adj Close day3'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        d.append(((x['Adj Close day4'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        e.append(((x['Adj Close day5'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        f.append(((x['Adj Close month'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        g.append(((x['Adj Close 6month'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        h.append(((x['Adj Close year'][z]-x['Price Per share'][z])/x['Price Per share'][z])*100)
        
    x['day1 return']=a
    x['day2 return']=b
    x['day3 return']=c
    x['day4 return']=d
    x['day5 return']=e
    x['month return']=f
    x['6month return']=g
    x['year return']=h
    
    return x

In [38]:
x=calculate_returns(x)


In [39]:
x_buy=x[x['Buy/Sell']=="Buy"]
x_buy=x_buy.reset_index()
x_sell=x[x['Buy/Sell']=="Sell"]
x_sell=x_sell.reset_index()

In [40]:
def preparing_for_boxplot(df,col1,col2,col3,col4,col5):
    a=[]
    b=[]
    for x in range(len(df[col1])):
        a.append(col1)
        b.append(df[col1][x])
        
    for x in range(len(df[col2])):
        a.append(col2)
        b.append(df[col2][x])
        
    for x in range(len(df[col3])):
        a.append(col3)
        b.append(df[col3][x])
        
    for x in range(len(df[col4])):
        a.append(col4)
        b.append(df[col4][x])
        
    for x in range(len(df[col5])):
        a.append(col5)
        b.append(df[col5][x])
        
    d=pd.DataFrame(a)
    d.columns=['day']
    d['return']=b
    
    return d

In [41]:
buy=preparing_for_boxplot(x_buy,'day1 return','day2 return','day3 return','day4 return','day5 return')
sell=preparing_for_boxplot(x_sell,'day1 return','day2 return','day3 return','day4 return','day5 return')

In [42]:

trace0 = go.Box(
    y=buy['return'],
    x=buy['day'],
    name='buy',
    marker=dict(
        color='#96a8ba'
    )
)
trace1 = go.Box(
    y=sell['return'],
    x=sell['day'],
    name='sell',
    marker=dict(
        color='#f1d6cc'
    )
)
    
data = [trace0, trace1]
    
layout = go.Layout(
    title='Short term returns on Insider Trades',
    yaxis=dict(
        title='Returns in %',
        zeroline=False
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [43]:
buy=buy[buy['return']<50]
sell=sell[sell['return']<50]

In [44]:
trace0 = go.Box(
    y=buy['return'],
    x=buy['day'],
    name='buy',
    marker=dict(
        color='#96a8ba'
    )
)
trace1 = go.Box(
    y=sell['return'],
    x=sell['day'],
    name='sell',
    marker=dict(
        color='#f1d6cc'
    )
)
    
data = [trace0, trace1]
    
layout = go.Layout(
    title='Short Term Returns on Insider Trades',
    yaxis=dict(
        title='Returns in %',
        zeroline=False
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

# Analyzing Long Term Returns

In [45]:
sp500_2=pd.read_csv('/data/sp500_2.csv')

In [46]:
sp500_2['Date']=pd.to_datetime(sp500_2['Date'],errors='coerce')
sp500_index=[x for x in sp500_2['Date'] if x.date()<datetime.date(2017,1,6)]
sp500_3=sp500_2[sp500_2['Date'].isin(sp500_index)]

In [47]:
def calculate_sp_returns(x):
    
    x['new_trans_date']=[time.strptime(str(y.date()), "%Y-%m-%d") for y in x['Date']]
    sp500_2.index=[time.strptime(str(x.date()), "%Y-%m-%d") for x in sp500_2['Date']]
    
    
    l,m,n,o,p,q,v,u,r=([] for i in range (9))
    for i in range(len(x)):
        for j in range(len(sp500_2)):
            if x['new_trans_date'][i]==sp500_2.index[j]:
                
                #q.append(sp500_2['Adj Close'][j])
                l.append(sp500_2['Adj Close'][j+1])
                m.append(sp500_2['Adj Close'][j+2])
                n.append(sp500_2['Adj Close'][j+3])
                o.append(sp500_2['Adj Close'][j+4])
                p.append(sp500_2['Adj Close'][j+5])
                r.append(sp500_2['Adj Close'][j+30])
                u.append(sp500_2['Adj Close'][j+180])
                v.append(sp500_2['Adj Close'][j+340])
                
    #x['Adj Close']=q
    x['Adj Close day1']=l
    x['Adj Close day2']=m
    x['Adj Close day3']=n
    x['Adj Close day4']=o
    x['Adj Close day5']=p
    x['Adj Close month']=r
    x['Adj Close 6month']=u
    x['Adj Close year']=v
    
    a,b,c,d,e,f,g,h=([] for k in range(8))
    for z in range(len(x)):
        a.append(((x['Adj Close day1'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        b.append(((x['Adj Close day2'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        c.append(((x['Adj Close day3'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        d.append(((x['Adj Close day4'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        e.append(((x['Adj Close day5'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        f.append(((x['Adj Close month'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        g.append(((x['Adj Close 6month'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        h.append(((x['Adj Close year'][z]-x['Adj Close'][z])/x['Adj Close'][z])*100)
        
    x['day1 return']=a
    x['day2 return']=b
    x['day3 return']=c
    x['day4 return']=d
    x['day5 return']=e
    x['month return']=f
    x['6month return']=g
    x['year return']=h
    
    return x
    

In [48]:
sp500_returns=calculate_sp_returns(sp500_3)

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

In [49]:
def prepare_longterm_boxplot(df,col1,col2,col3):
    a=[]
    b=[]
    for x in range(len(df[col1])):
        a.append(col1)
        b.append(df[col1][x])
        
    for x in range(len(df[col2])):
        a.append(col2)
        b.append(df[col2][x])
        
    for x in range(len(df[col3])):
        a.append(col3)
        b.append(df[col3][x])
        
    d=pd.DataFrame(a)
    d.columns=['day']
    d['return']=b
    
    return d

In [50]:
long_buy=prepare_longterm_boxplot(x_buy,'month return','6month return','year return')
long_sell=prepare_longterm_boxplot(x_sell,'month return','6month return','year return')

In [51]:
sp_box=prepare_longterm_boxplot(sp500_returns,'month return','6month return','year return')

In [52]:
trace0 = go.Box(
    y=long_buy['return'],
    x=long_buy['day'],
    name='buy',
    marker=dict(
        color='#35e3c4'
    )
)
trace1 = go.Box(
    y=long_sell['return'],
    x=long_sell['day'],
    name='sell',
    marker=dict(
        color='#172d2b'
    )
)
trace2 = go.Box(
    y=sp_box['return'],
    x=sp_box['day'],
    name='sp500',
    marker=dict(
        color='#6da77a'
    )
)
    
data = [trace0, trace1,trace2]
    
layout = go.Layout(
    title='Long term returns on Insider Trades and SP500 stocks',
    yaxis=dict(
        title='Returns in %',
        zeroline=False
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [53]:
long_buy=long_buy[long_buy['return']<100]
long_sell=long_sell[long_sell['return']<100]
sp_box=sp_box[sp_box['return']<100]

In [54]:
trace0 = go.Box(
    y=long_buy['return'],
    x=long_buy['day'],
    name='buy',
    marker=dict(
        color='#35e3c4'
    )
)
trace1 = go.Box(
    y=long_sell['return'],
    x=long_sell['day'],
    name='sell',
    marker=dict(
        color='#172d2b'
    )
)
trace2 = go.Box(
    y=sp_box['return'],
    x=sp_box['day'],
    name='sp500',
    marker=dict(
        color='#6da77a'
    )
)
    
data = [trace0, trace1,trace2]
    
layout = go.Layout(
    title='Long Term Returns on Insider trades and SP500 Stocks',
    yaxis=dict(
        title='Returns in %',
        zeroline=False
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

#  Analyzing triads in Insider Trading patterns for Facebook

In [55]:
fb=df[df['Issuer Name']=='Facebook Inc']

In [56]:
fb=fb.reset_index()
fb=fb.drop(labels=['index',"Unnamed: 0"],axis=1)

In [57]:
o=[]
for x in range(len(fb)-1):
    i=[]
    if(fb['Buy/Sell'][x]=='Buy' and fb['Buy/Sell'][x+1]=="Sell"):
        if(fb['Price Per share'][x]<fb['Price Per share'][x+1]):
            i.append(x)
            i.append(x+1)
            o.append(i)
        else:
            break
    elif(fb['Buy/Sell'][x]=='Sell' and fb['Buy/Sell'][x+1]=="Buy"):
        if(fb['Price Per share'][x]>fb['Price Per share'][x+1]):
            i.append(x)
            i.append(x+1)
            o.append(i)
        else:
            break
    
        
            
        

In [58]:
l=[item for sublist in o for item in sublist]
f=list(set(l))
f.sort()

In [59]:
index=[x for x in range(3)]
columns=['first_date','first_incident','first_volume','first_price','second_date','second_incident','second_volume','third_date','third_incident','third_volume','third_price']
insider_pattern = pd.DataFrame(index=index, columns=columns)
insider_pattern = insider_pattern.fillna(0)

In [60]:
insider_pattern['first_incident']=['Sell_1',fb['Buy/Sell'][14], 'Sell_2']
insider_pattern['first_date']=[fb['Transaction Date'][13],fb['Transaction Date'][14],fb['Transaction Date'][15]]
insider_pattern['first_volume']=[fb['Transcation Share'][13],fb['Transcation Share'][14],fb['Transcation Share'][15]]
insider_pattern['first_price']=[fb['Price Per share'][13],-fb['Price Per share'][14],fb['Price Per share'][15]]

insider_pattern['second_incident']=["Sell_1",fb['Buy/Sell'][40],"Sell_2"]
insider_pattern['second_date']=[fb['Transaction Date'][39],fb['Transaction Date'][40],fb['Transaction Date'][41]]
insider_pattern['second_volume']=[fb['Transcation Share'][39],fb['Transcation Share'][40],fb['Transcation Share'][41]]
insider_pattern['second_price']=[fb['Price Per share'][39],-fb['Price Per share'][40],fb['Price Per share'][41]]

insider_pattern['third_incident']=["Sell_1",fb['Buy/Sell'][106],"Sell_2"]
insider_pattern['third_date']=[fb['Transaction Date'][105],fb['Transaction Date'][106],fb['Transaction Date'][107]]
insider_pattern['third_volume']=[fb['Transcation Share'][105],fb['Transcation Share'][106],fb['Transcation Share'][107]]
insider_pattern['third_price']=[fb['Price Per share'][105],-fb['Price Per share'][106],fb['Price Per share'][107]]



In [61]:
trace0=go.Bar(
x=insider_pattern['first_incident'],
y=insider_pattern['first_price'],
name='Price Per Share',
marker=dict(color='#62704d '),
)

trace1=go.Bar(
x=insider_pattern['first_incident'],
y=insider_pattern['first_price']*insider_pattern['first_volume'],
name='Total Value',
marker=dict(color='#846e50 '))

trace2=go.Bar(
x=insider_pattern['second_incident'],
y=insider_pattern['second_price'],
name='Price Per Share',
marker=dict(color='#b85941 '))

trace3=go.Bar(
x=insider_pattern['second_incident'],
y=insider_pattern['second_price']*insider_pattern['second_volume'],
name='Total Value',
marker=dict(color='#625643 '))

fig = tools.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{}, {}]],
                          subplot_titles=('                                             Incident at 2016-12-13 ',' ', '                                             Incident at 2016-11-11',''))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)
fig.append_trace(trace3, 2, 2)

fig['layout']['xaxis2'].update(title='Net Profit:981,508.2 $')

fig['layout']['xaxis4'].update(title='Net Profit:4,352,792 $')
fig['layout']['yaxis1'].update(title='Price Per Share')
fig['layout']['yaxis2'].update(title='Total Value')
fig['layout']['yaxis3'].update(title='Price Per Share')
fig['layout']['yaxis4'].update(title='Total Vaue')
fig['layout'].update( height=900,width=700,paper_bgcolor = "#dbd8a7",showlegend=False,title='Facebook Insider Triads')
iplot(fig)


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

